# TensorFlow Regression Example

More Realistic. More data points. Batches.

The tf.estimator is for things that are easier. TensorFlow is more for things that need a specific neural network, customized, whatever...

## Imports

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf

from sklearn.model_selection import train_test_split

# remember TensorFlow and SciKit-Learn up here

## Creating Data

#### One Million Points!

In [ ]:
x_data = np.linspace(0.0, 10.0, 1000000) # We're not quite ready for a real dataset

In [ ]:
x_data

#### Noise

In [ ]:
# No random seed (?)
noise = np.random.randn(len(x_data))

In [ ]:
noise

#### Now, for the data

$ y = mx + b + noise $ just to make it more difficult for the model

Jose, seemingly arbitrarily, chooses $ b = 5 $ and $ m = 0.5 $ to start

In [ ]:
y_true = ( 0.5 * x_data ) + 5 + noise

Pandas

In [ ]:
x_df = pd.DataFrame(data=x_data, columns=['X Data'])

In [ ]:
x_df.head()

In [ ]:
y_df = pd.DataFrame(data=y_true, columns=['Y'])

In [ ]:
y_df.head()

In [ ]:
my_data = pd.concat(
         [ x_df, y_df], axis=1) 
              # axis=1 keeps it from stacking on like pancakes

Copied from the course notes version:

```
my_data = pd.concat(
        [pd.DataFrame(data=x_data,columns=['X Data']),
         pd.DataFrame(data=y_true,columns=['Y'])
    ],
    axis=1
)
```

In [ ]:
my_data.head()

In [ ]:
# my_data.plot() might crash the kernel
my_sample = my_data.sample(n=250)

In [ ]:
my_sample.head()

In [ ]:
my_sample.plot(kind='scatter', x = 'X Data', y='Y')

# TensorFlow

## Batch Size

> We will take the data in batches (1 000 000 points is a lot to pass in at once).

In [ ]:
# random points to grab, If you had a trillion, probably use smaller batches
batch_size = 8

#### Variables

In [ ]:
m_pre, b_pre = np.random.randn(2)

In [ ]:
type(m_pre)

In [ ]:
type(b_pre)

In [ ]:
# I didn't follow this, because using 'dtype' instead of 'type' worked
#tf.cast(m_pre, tf.float32)
#tf.cast(b_pre, tf.float32)

In [ ]:
#  DWB, I had to add the type
#+ Jose just used, e.g. 'm = tf.Variable(0.81)'
m = tf.Variable(m_pre, dtype=tf.float32) 
b = tf.Variable(b_pre, dtype=tf.float32)

print("Initally: m = " + str(m_pre) + " ; " + "b = " + str(b_pre))

#### Placeholders

In [ ]:
x_ph = tf.placeholder(tf.float32, [batch_size])

In [ ]:
y_ph = tf.placeholder(tf.float32, [batch_size])

So, I'm getting that placeholders get your data, while variables are what you're trying to predict. I'm not sure that's exactly correct, but it's what I'm getting right now.

#### Graph
 
What are we trying to do here? Fit a line to some points. So it's a $ y = mx + b $ kind of graph

In [ ]:
y_model = m * x_ph + b # Had to mess with type to get this to work

#### Loss Function

In [ ]:
#  Remember that y_value is the true value
#+ Also, we square it to punish the error more,
#+ and thus bring it closer more quickly.
#+ could use '() ** 2' instead of tf.square()

error = tf.reduce_sum(tf.square(y_ph - y_model))

#### Optimizer

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
train = optimizer.minimize(error)

#### Initialize Variables

In [ ]:
init = tf.global_variables_initializer()

### Session

In [ ]:
with tf.Session() as sess:
    
    sess.run(init)
    
    batches = 1000
    
    for i in range(batches):
        
        rand_index = np.random.randint(len(x_data),
                                       size=batch_size)
        
        #  DWB: it seems to me we'll only we doing 8000 out
        #+ of the 1e6 points. That will make it go faster, I
        #+ guess.
        #
        #  Jose says we can play around with batches and
        #+ batch_size to see if we have enough data to 
        #+ train it well. He seems to suggest that, if we
        #+ were to use more of the training data, we would
        #+ overfit to the training data. Not sure if that
        #+ applies here ... wait, yes it kinda does but not
        #+ in a way that's too concerning - we're taking
        #+ random parts ...
        
        feed = {x_ph:x_data[rand_index], 
                y_ph:y_true[rand_index]}
                
        sess.run(train, feed_dict=feed)
        
        #  So, we have it fitting the data with 8 random points
        #+ for each
        
    ##endof:  for i
    
    #  Fetch the slope and intercept values (run will go get the 
    #+ m and b placeholders)
    model_m, model_b = sess.run([m, b])
    
##endof:  with ... sess

In [ ]:
model_m # should come out close to our 0.5

In [ ]:
model_b #should come out close to our 5

So, we went from whatever our original m and b values were - in my case $ m = -1.8 $ and $ b = 0.5 $. The values used for this specific training can be found with the following cell.

In [ ]:
print("m_init = " + str(m_pre) + " ; " + "b_init = " + str(b_pre))

And we ended up with the `model_m` and `model_b` shown above, which are quite close to the values before noise, m = 0.5, b = 5; Things would look even nicer if we took the error over the value.

In [ ]:
print("Delta_m = " + str(abs(0.5 - model_m)) + ";\n" + \
      "Delta_b = " + str(abs(5.0 - model_b)))

### Results

In [ ]:
y_hat = x_data * model_m + model_b # rem. y_hat represents the predicted

In [ ]:
my_data.sample(250).plot(kind="scatter",
                         x="X Data", y="Y")
plt.plot(x_data, y_hat, 'r') #  Oh, so I see Pandas is using 
                             #+ the matplotlib canvas. Cool!

Jose changes the above code for 10k batches, I'm going to have it all re-written, so I can compare better. I will stick it to the anti-Q&R voice by not renaming the variables. Wahahaha!

I though I might have to rename them, then I think I figured that I could get rid of an error that came up by initializing the variables. Nope, had to re-put-in all the code. But I'm not renaming the variables. Wahahaha!

In [ ]:
#  DWB, I had to add the type
#+ Jose just used, e.g. 'm = tf.Variable(0.81)'
m = tf.Variable(m_pre, dtype=tf.float32) 
b = tf.Variable(b_pre, dtype=tf.float32)
print("Initally: m = " + str(m_pre) + " ; " + "b = " + str(b_pre))
x_ph = tf.placeholder(tf.float32, [batch_size])
y_ph = tf.placeholder(tf.float32, [batch_size])
y_model = m * x_ph + b
error = tf.reduce_sum(tf.square(y_ph - y_model))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
train = optimizer.minimize(error)
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    
    sess.run(init)
    
    batches = 10000
    
    for i in range(batches):
        
        rand_index = np.random.randint(len(x_data),
                                       size=batch_size)
        
        feed = {x_ph:x_data[rand_index], 
                y_ph:y_true[rand_index]}
                
        sess.run(train, feed_dict=feed)
        
    ##endof:  for i
    
    #  Fetch the slope and intercept values (run will go get the 
    #+ m and b placeholders)
    model_m, model_b = sess.run([m, b])
    
##endof:  with ... sess

In [ ]:
model_m

In [ ]:
model_b

After re-puttting-in the code, I got my answers.

In [ ]:
print("m_init = " + str(m_pre) + " ; " + "b_init = " + str(b_pre))
print("m_final = " + str(model_m) + " ; " + "b_fin = " + str(model_b))

print("Delta_m = " + str(abs(0.5 - model_m)) + ";\n" + \
      "Delta_b = " + str(abs(5.0 - model_b)))
print()
print("Hmmm ...")
print()
print("Compare to:" + '\n' + \
      "Delta_m = 0.006303846836090088" + '\n' + "and" + \
      '\n' + "Delta_b = 0.055045127868652344" + '\n' +  \
      "for 8000 batches." + \
      '\n\n' + "... interesting ...")

In [ ]:
y_hat = x_data * model_m + model_b
my_data.sample(250).plot(kind="scatter",
                         x="X Data", y="Y")
plt.plot(x_data, y_hat, 'r')

Jose stated that the noise might make it so they might not be so different.

He noted (as I'd been thinking) that we haven't been doing the train/test split. We will with `tf.estimator`

## tf.estimator API

> Much simpler API for basic tasks like regression! We'll talk about more abstractions like TF-Slim later on.

### Types

```
tf.estimator.LinearClassifier
tf.estimator.LinearRegressor
tf.estimator.DNNClassifier
tf.estimator.DNNRegressor
```

Jose says `DNN` is for Densely-connected Neural Network. I'm not sure that it's not Deep, but I am leaning towards thinking he's right.

Combined-type

```
tf.estimator.DNNLinearCombinedClassifier
tf.estimator.DNNLinearCombinedRegressor
```

### Steps

- Define a list of feature columns
- Create the Estimator Model
- Create a Data Input Function
- Call `train`, `evaluate`, and `predict` on the object

#### We're going to use maybe-not-the-best use case, with just one feature, but it will get us the idea

In [ ]:
# deeper dive into feature column later
feat_cols = [ tf.feature_column.numeric_column('x', shape=[1]) ]

In [ ]:
estimator = tf.estimator.LinearRegressor(feature_columns=feat_cols)

Note the output

```
INFO:tensorflow:Using default config.
WARNING:tensorflow:Using temporary folder as model directory: C:\Users\Anast\AppData\Local\Temp\tmpn9_bnvpm
INFO:tensorflow:Using config: {'_session_config': None, '_device_fn': None, '_keep_checkpoint_max': 5, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002E2998E5128>, '_task_type': 'worker', '_global_id_in_cluster': 0, '_num_worker_replicas': 1, '_log_step_count_steps': 100, '_is_chief': True, '_train_distribute': None, '_save_checkpoints_secs': 600, '_task_id': 0, '_service': None, '_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_master': '', '_tf_random_seed': None, '_evaluation_master': '', '_save_checkpoints_steps': None, '_model_dir': 'C:\\Users\\Anast\\AppData\\Local\\Temp\\tmpn9_bnvpm'}
```

with probably a different string at the end of the path each time.

### Train Test Split

> We haven't actually performed a train test split yet! So let's do that on our data now and perform a more realistic version of a Regression Task

In [ ]:
# type 'train_test_split', then do [Shift] + [Tab] to get the line
x_train, x_eval, y_train, y_eval = \
                train_test_split(x_data, y_true, 
                                 test_size=0.3, 
                                 random_state=101) # match Jose

In [ ]:
print("x_train.shape = " + str(x_train.shape))
print("y_train.shape = " + str(y_train.shape))
print()
print("x_eval.shape = " + str(x_eval.shape))
print("y_eval.shape = " + str(y_eval.shape))

### Set up Estimator Inputs

> ... input_function that kind of serves like your feed dictionary and your batch size indicator ...

Jose

In [ ]:
# Can also do .pandas_input_fn if you're coming from a Pandas dataframe
input_func = \
    tf.estimator.inputs.numpy_input_fn({'x':x_train},
                                       y_train,
                                       batch_size=4,
                                       num_epochs=None,
                                       shuffle=True
    )

In [ ]:
train_input_func = \
    tf.estimator.inputs.numpy_input_fn({'x':x_train},
                                       y_train,
                                       batch_size=4,
                                       num_epochs=1000,
                                       shuffle=False
    )
#  Why shuffle is False?
#+ Jose says, "And I also wanna set shuffle equal to false.
#+            "Okay, there we go.
#+            "And the reason I have a shuffle equals false 
#+            "here for this
#+            "train is because I'm gonna be using this 
#+            "train input
#+            "function for evaluation against a test input 
#+            "function."
#+ DWB:  The one just below, eval_input_function, which also
#+ DWB: has shuffle=False
#+ DWB:  Not sure I understand this, but let's go with it.

In [ ]:
eval_input_func = \
    tf.estimator.inputs.numpy_input_fn({'x':x_eval},
                                       y_eval,
                                       batch_size=4,
                                       num_epochs=1000,
                                       shuffle=False
    )

### Train the Estimator

In [ ]:
estimator.train(input_fn=input_func, steps=1000)
    #  We do  steps=1000 , since we didn't specify training
    #+ epochs for our  input_func .

### Evaluation

In [ ]:
train_metrics = estimator.evaluate(input_fn=train_input_func,
                                   steps=1000
)

In [ ]:
eval_metrics = estimator.evaluate(input_fn=eval_input_func,
                                  steps=1000
)

In [ ]:
print("train metrics: {}".format(train_metrics)) # Python 3.5 allows this form
print("eval metrics : {}".format(eval_metrics))
print("The loss is pretty close for both, which " + \
      "\nis a good sign that we're not overfitting")

### Predictions

In [ ]:
brand_new_data = np.linspace(0, 10, 10)
input_fn_predict = \
    tf.estimator.inputs.numpy_input_fn({'x':brand_new_data},
                                       shuffle=False
    )

In [ ]:
estimator.predict(input_fn=input_fn_predict)

In [ ]:
# Easy to see the output of the generator by casting it as a list
list(estimator.predict(input_fn=input_fn_predict))

In [ ]:
# Repeat, for size
my_list = list(estimator.predict(input_fn=input_fn_predict))
print("\nThere are " + str(len(my_list)) + " entries," + \
      "\nas we expect (as long as there are 10 entries).")

In [ ]:
predictions = [] #could also use  np.array[]
for x in estimator.predict(input_fn=input_fn_predict):
    predictions.append(x['predictions'])

In [ ]:
predictions # how did we do?

In [ ]:
my_data.sample(n=250).plot(kind='scatter',
                           x='X Data', y='Y')
plt.plot(brand_new_data, predictions, 'r')

In [ ]:
my_data.sample(n=250).plot(kind='scatter',
                           x='X Data', y='Y')
plt.plot(brand_new_data, predictions, 'r*')

Jose usually says

# Great Job!

around here, which I think is cool.

_That's all for now!_